In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightkurve as lk
import sys
import os
import ast




In [4]:
import os
import sys

# Add the project root (two levels up from ~/tutorials/detrend_tutorial) to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
package_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir, 'democratic_detrender'))

sys.path.insert(0, project_root)
sys.path.insert(0, package_root)

from get_lc import *


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [5]:
#find the TIC ID for another catalog star name
tic_id = tic_id_from_simbad('Wasp-63')

In [6]:
#pull in transit metadata from exoplanet archive (period, TO, duration)
transit_info = transit_info_from_exoplanet_archive(tic_id)
print(transit_info)

mask_width = 1.1 #assume a mask width of 1.1 times the duration of the transit
periods = np.array(transit_info["period [days]"].values, dtype=float)
t0s = np.array(transit_info["t0 [BJD]"].values, dtype=float)
durations = np.array(transit_info["duration [hours]"].values, dtype=float)



# switch to TESS BJD
t0s = t0s - 2457000

Exoplanet Archive: Planet Comparison Table
------------------------------------------
             tic_id      t0 [BJD]  period [days]  duration [hours]
4209  TIC 393414358  2.458575e+06        4.37808          5.390098


In [7]:
# pull in short cadence TESS SPOC LC
lc_files = lk.search_lightcurve(
    tic_id, mission="TESS", author="SPOC", cadence="short"
).download_all(quality_bitmask="default", flux_column='pdcsap_flux')

# pull in the quarters start and end times, crowding, and flux_fraction values for each quarter
quarters = []
crowding = []
flux_fraction = []
for file in lc_files:
    quarters.append([np.min(file.time.value), np.max(file.time.value)])
    crowding.append(file.CROWDSAP)
    flux_fraction.append(file.FLFRCSAP)

In [8]:
#stitch the quarters togethers
lc = lc_files.stitch().remove_nans()


xs = lc.time.value
ys = lc.flux
ys_err = lc.flux_err

In [9]:
#for Wasp-63, there is only 1 planet
nplanets = 1
planet_number = 1

mask = np.zeros(np.shape(xs), dtype=bool)
for ii in range(0, nplanets):
    masks = lc.create_transit_mask(
        period=periods[ii],
        duration=durations[ii] / 24.0 * mask_width,
        transit_time=t0s[ii],
    )
    mask += masks

mask_fitted_planet = lc.create_transit_mask(
    period=periods[planet_number - 1],
    duration=durations[planet_number - 1] / 24.0 * mask_width,
    transit_time=t0s[planet_number - 1],
)

In [10]:
# save the period, duration, and t0 for the planet we are fitting for...
period = np.array([periods[planet_number - 1]])
t0 = t0s[planet_number - 1]
duration = np.array([durations[planet_number - 1]])

print("using the following params for the planet we are fitting")
print("--------------------------------------------------------")
print("[  t0 [TESS BJD]  , P [days], tdur [hrs]")
print("[" + str(t0) + ", " + str(period[0]) + ", " + str(duration[0]) + "]")

using the following params for the planet we are fitting
--------------------------------------------------------
[  t0 [TESS BJD]  , P [days], tdur [hrs]
[1574.7710199998692, 4.37808, 5.3900978]


In [11]:
#create transit masks and 
min_time = xs.min()
max_time = xs.max()
t0s_all = []
while t0 > min_time:
    t0 -= period[0]

while t0 < max_time:
    t0s_all.append(t0)
    #print(t0s_all)
    t0 += period[0]

cadence = determine_cadence(xs)

t0s_in_data = []
for t0 in t0s_all:
    nearest_lc_time = find_nearest(xs, t0)

    # if there is a data point within the cadence (times some uncertainty lets say 3) of
    # expected transit then there should be transit data
    if np.abs(t0 - nearest_lc_time) < 3 * cadence:
        t0s_in_data.append(t0)
        
        
        
print(str(len(t0s_in_data)) + " transits (or epochs) in total")


26 transits (or epochs) in total


In [12]:
#normalize the data
mu = np.median(ys)
ys = ys / mu - 1
ys_err = ys_err / mu

In [13]:
# Creating the DataFrame
lc = pd.DataFrame({
    'xs': np.array(xs),
    'ys': np.array(ys),
    'ys_err': np.array(ys_err),
    'mask': mask,
    'mask_fitted_planet': mask_fitted_planet
})

lc

,xs,ys,ys_err,mask,mask_fitted_planet
0,1468.274705,-0.000042,0.001268,False,False
1,1468.276093,0.000075,0.001268,False,False
2,1468.277482,0.001087,0.001270,False,False
3,1468.278871,0.002190,0.001268,False,False
4,1468.280260,-0.001718,0.001268,False,False
...,...,...,...,...,...
80453,3689.917611,-0.001423,0.001305,False,False
80454,3689.924555,0.003232,0.001307,False,False
80455,3689.927333,0.000891,0.001308,False,False
80456,3689.938444,0.002021,0.001311,False,False


In [14]:
lc_metadata = pd.DataFrame({
    'quarters': quarters,
    'crowding': crowding,
    'flux_fraction': flux_fraction
})

lc_metadata

,quarters,crowding,flux_fraction
0,"[1468.2747045698593, 1490.04695665661]",0.998463,0.894397
1,"[1491.634441835873, 1516.0881598528044]",0.997194,0.913473
2,"[2174.2308321088685, 2200.2341139154987]",0.997357,0.914399
3,"[2201.7355179644114, 2227.574340017412]",0.998423,0.901020
4,"[3663.044084605974, 3689.939832482194]",0.997572,0.904219


In [15]:
orbital_data = pd.DataFrame({
    't0': t0,
    'period': period,
    'duration': duration,
})

orbital_data   

,t0,period,duration
0,3689.38366,4.37808,5.390098


In [16]:
t0s_output = pd.DataFrame({
    't0s_in_data': t0s_in_data
})

t0s_output

,t0s_in_data
0,1469.69710
1,1474.07518
2,1478.45326
3,1482.83134
4,1487.20942
5,1495.96558
6,1500.34366
7,1509.09982
8,1513.47790
9,2174.56798


In [17]:
# Save the DataFrames as CSVs file

lc.to_csv('lc.csv', index=False)
lc_metadata.to_csv('lc_metadata.csv', index=False)
orbital_data.to_csv('orbital_data.csv', index=False)
t0s_output.to_csv('t0s.csv', index=False)


In [34]:
orbit_test = pd.read_csv('orbital_data.csv')
orbit_test['duration'][0]

5.3900978

In [22]:
t0s_in_data

[1469.6970999998703,
 1474.0751799998702,
 1478.4532599998702,
 1482.8313399998701,
 1487.20941999987,
 1495.96557999987,
 1500.34365999987,
 1509.0998199998699,
 1513.4778999998698,
 2174.567979999863,
 2178.946059999863,
 2183.324139999863,
 2187.702219999863,
 2192.0802999998627,
 2196.4583799998627,
 2205.2145399998626,
 2209.5926199998626,
 2218.3487799998625,
 2222.7268599998624,
 2227.1049399998624,
 3667.4932599998474,
 3671.8713399998474,
 3676.2494199998473,
 3680.6274999998473,
 3685.005579999847,
 3689.383659999847]